## Load GeoJSON from Bane NOR API
This script extracts map info from Bane NORs publicly accessible train map database, and writes it out to an excel file, to be imported into power apps.
Some notes:
* This is built with some simple thoughts on modularization, by using separate functions.
* Pandas is used to store and process the data. Openpyxl is neccessary to write the data to excel. 
* Requests is used to fetch the data.
* It is possible to only get track location. The default is to get both track location AND station locations.
* Power apps enforces a limit of max 500 GeoJSON objects in any source. 
As the total we are getting is 540 rows, the last 40 is simply discarded. No tracks are lost, but some stations are lost. Deal with it. 

In [ ]:
saveFileName = "GeoJSON_tracks"

# DO you want to load stations as well?
trainStations = True
convertStationsToCircles = True

# Enforce the power apps limit of 500
truncate_down_to_limit = True
truncate_limit = 500

#API ingest settings
offset = 0
limit = 2000

In [2]:
# Import setup
try:
    import pandas as pd
except ImportError:
    import sys
    !{sys.executable} -m pip install openpyxl
    import pandas as pd

try:
    import openpyxl
except ImportError:
    import sys
    !{sys.executable} -m pip install openpyxl


try:
    import requests
except ImportError:
    import sys
    !{sys.executable} -m pip install requests
    import requests

Defining the function that gets the data from the API.

In [ ]:
def getMapResponse(offset, trainStations = False, **kwargs):
    '''Returns a JSON-reponse object containing limit (default and max is 2000) number of items, starting from offset. trainStations determines whether or not the function returns stations or tracks.'''

    limit = kwargs.get("limit", 2000)
    inputalternativ = kwargs.get("inputalternativ", 1) # 1 is default.

    if trainStations:
        #Denne inneholder stasjoner, og brukes sammen med alt 1. 
        url = f"https://dvlkart.banenor.no/arcgis/rest/services/Banenettverk/MapServer/2/query?where=1%3D1&text=&objectIds=&time=&timeRelation=esriTimeRelationOverlaps&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&distance=&units=esriSRUnit_Meter&relationParam=&outFields=Navn&returnGeometry=true&returnTrueCurves=false&maxAllowableOffset=&geometryPrecision=&outSR=&havingClause=&returnIdsOnly=false&returnCountOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&gdbVersion=&historicMoment=&returnDistinctValues=false&resultOffset={offset}&resultRecordCount={limit}&returnExtentOnly=false&sqlFormat=none&datumTransformation=&parameterValues=&rangeValues=&quantizationParameters=&featureEncoding=esriDefault&f=geojson"
    elif inputalternativ == 1:
        #Goldilocks principle -> perfect! Contains a simplified model with lower resolution. 
        url = f"https://dvlkart.banenor.no/arcgis/rest/services/Banenettverk/MapServer/10/query?where=1%3D1&text=&objectIds=&time=&timeRelation=esriTimeRelationOverlaps&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&distance=&units=esriSRUnit_Meter&relationParam=&outFields=Banenavn&returnGeometry=true&returnTrueCurves=false&maxAllowableOffset=&geometryPrecision=&outSR=&havingClause=&returnIdsOnly=false&returnCountOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&gdbVersion=&historicMoment=&returnDistinctValues=false&resultOffset={offset}&resultRecordCount={limit}&returnExtentOnly=false&sqlFormat=none&datumTransformation=&parameterValues=&rangeValues=&quantizationParameters=&featureEncoding=esriDefault&f=geojson"
    elif inputalternativ == 2:
        #Denne inneholder 243 rader, men hver inneholder for lange objekter til å vises korrekt, så de blir croppet av, og viser bare starten på strekninger.
        url = f"https://dvlkart.banenor.no/arcgis/rest/services/Banelenke/MapServer/0/query?where=1%3D1&text=&objectIds=&time=&timeRelation=esriTimeRelationOverlaps&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&distance=&units=esriSRUnit_Foot&relationParam=&outFields=*&returnGeometry=true&returnTrueCurves=false&maxAllowableOffset=&geometryPrecision=&outSR=&havingClause=&returnIdsOnly=false&returnCountOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&gdbVersion=&historicMoment=&returnDistinctValues=false&resultOffset={offset}&resultRecordCount={limit}&returnExtentOnly=false&sqlFormat=none&datumTransformation=&parameterValues=&rangeValues=&quantizationParameters=&featureEncoding=esriDefault&f=geojson"
    elif inputalternativ == 3:
        #Denne inneholder for mange rader, som hver er ganske korte, og blir for mange til at Powerapps kan vise dem korrekt. Den viser bare de første 2000.
        url = f"https://dvlkart.banenor.no/arcgis/rest/services/Banenettverk/MapServer/14/query?where=1%3D1&text=&objectIds=&time=&timeRelation=esriTimeRelationOverlaps&geometry=&geometryType=esriGeometryPolygon&inSR=&spatialRel=esriSpatialRelIntersects&distance=&units=esriSRUnit_Foot&relationParam=&outFields=Banenavn&returnGeometry=true&returnTrueCurves=false&maxAllowableOffset=&geometryPrecision=&outSR=&havingClause=&returnIdsOnly=false&returnCountOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&gdbVersion=&historicMoment=&returnDistinctValues=false&resultOffset={offset}&resultRecordCount={limit}&returnExtentOnly=false&sqlFormat=none&datumTransformation=&parameterValues=&rangeValues=&quantizationParameters=&featureEncoding=esriDefault&f=geojson"
    
    payload = {}
    headers = {}
    response = requests.get(url, headers=headers, data=payload)
    #print("URL used: " + url)
    print("Length of API response:", len(response.text))
    return response

Main loop: 
This function fills the dataframe SporDF with all the data from the API.

In [4]:
def fillMapDF(trainStations, offset, limit, df):
    hentetAlt = False
        
    while not hentetAlt:
        SporRawJSON = getMapResponse(offset = offset, trainStations = trainStations).json()  
        
        tempdf = pd.json_normalize(SporRawJSON)
        tempdf = tempdf.explode("features")
        print("Rows of new data:", tempdf.shape[0])
        df = pd.concat([df, tempdf], ignore_index=True)
        #print(sporDF.shape)

        if "exceededTransferLimit" in SporRawJSON:
            if SporRawJSON["exceededTransferLimit"] == True:
                print(f"Transfer limit exceeded. Objects number {offset}-{offset+limit} has been collected. Fetching the next {limit}...")
                offset += limit
        else:
            print("All data collected.")
            hentetAlt = True                
                    
    print("\n")
    return df

In [5]:
#Sample response
'''
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "geometry": {
        "type": "LineString",
        "coordinates": [
          [13.6299147114066, 66.1564617390492],
          [13.6298764104629, 66.1564525877301],
          [13.6299147114066, 66.1564617390492]
        ]
      },
      "properties": {
        "Banenavn": "Nordlandsbanen"
      }
    },
    {
      "type": "Feature",
      "geometry": {
        "type": "LineString",
        "coordinates": [
          [5.73934403181721, 60.4593989800107],
          [5.73934270707218, 60.4593853650686],
          [5.73934512720757, 60.4593612516533],
          [5.73932950205534, 60.4592613030111],
          [5.73930664342178, 60.4591609077203],
          [5.73925735839471, 60.4590101723198],
          [5.73919006967426, 60.4588611528486],
          [5.73910527704731, 60.4587143108161],
          [5.73901556669164, 60.4585875023944]
        ]
      },
      "properties": {
        "Banenavn": "Bergensbanen"
      }
    }
  ]
  '''

'\n  "type": "FeatureCollection",\n  "features": [\n    {\n      "type": "Feature",\n      "geometry": {\n        "type": "LineString",\n        "coordinates": [\n          [13.6299147114066, 66.1564617390492],\n          [13.6298764104629, 66.1564525877301],\n          [13.6299147114066, 66.1564617390492]\n        ]\n      },\n      "properties": {\n        "Banenavn": "Nordlandsbanen"\n      }\n    },\n    {\n      "type": "Feature",\n      "geometry": {\n        "type": "LineString",\n        "coordinates": [\n          [5.73934403181721, 60.4593989800107],\n          [5.73934270707218, 60.4593853650686],\n          [5.73934512720757, 60.4593612516533],\n          [5.73932950205534, 60.4592613030111],\n          [5.73930664342178, 60.4591609077203],\n          [5.73925735839471, 60.4590101723198],\n          [5.73919006967426, 60.4588611528486],\n          [5.73910527704731, 60.4587143108161],\n          [5.73901556669164, 60.4585875023944]\n        ]\n      },\n      "properties": 

### Populate the dataframe with track and station position data.

In [6]:
tracksDF = pd.DataFrame(None)

tracksDF = fillMapDF(trainStations=False, offset = offset, limit = limit, df = tracksDF)
if trainStations:
    finaldf = fillMapDF(trainStations=True, offset = offset, limit = limit, df = tracksDF)

print(f"Total number of JSON objects: {finaldf.shape[0]}.")

first_run_done = False # Used for debugging and backup later.

Length of API response: 134775
Rows of new data: 44
All data collected.


Length of API response: 65997
Rows of new data: 496
All data collected.


Total number of JSON objects: 540.


Function for extracting label ("name") for each GeoJSON object type, "Banenavn" for track, "Navn" for stations.

In [7]:
def setLabelTrackOrStationTest(row):
    if row["geometry"]["type"] == "LineString":
        #print("inside linestring: ", row.get("properties", None).get("Banenavn", "Ukjent banestrekning"))
        return row.get("properties", "L1").get("Banenavn", "Ukjent banestrekning")
    elif row["geometry"]["type"] == "Point":
        #print("inside point", row.get("properties", None).get("Navn", "Ukjent stasjon"))
        return row.get("properties", "P1").get("Navn", "Ukjent stasjon")
    elif row["geometry"]["type"] == "MultiLineString":
        return row.get("properties", "M1").get("Banenavn", "Ukjent banestrekning")

Converting all station points to a circle, by using the fact that GeoJSON circles are supported as type: Point with a subtype: "Circle" and radius: xx

In [8]:
def makeCircleFromPoint(row):
    if row["geometry"]["type"] == "Point":
        row["properties"]["subType"] = "Circle"
        row["properties"]["radius"] = 100
    return row

Extracting "banenavn" (track name) from properties, and creating a separate column for that. Also adding a a columns where features is converted to string, to enable string.replace-functions. THat is needed to replace ' to ". Also adding a column with len(strfeature) for fun
.

In [9]:
#Purely for debugging the last part
if not first_run_done:
    first_run_done = True
    print("This is the first run. Backup initialized.")
    finaldf_backup = finaldf.copy()

else:
    finaldf = finaldf_backup.copy()
    print("This is a subsequent run. Using backup to fill finaldf")


This is the first run. Backup initialized.


In [10]:
#sporDF = sporDF.assign(label2 = sporDF["features"].apply(lambda x: x["geometry"]["type"])) 
finaldf.index.name = "index"
finaldf = finaldf.assign(label = finaldf["features"].apply(lambda x: setLabelTrackOrStationTest(x)))
if convertStationsToCircles:
    finaldf["features"] = finaldf["features"].apply(lambda x: makeCircleFromPoint(x))


finaldf = finaldf.assign(strfeatures = finaldf["features"].astype("string").str.replace("'", '"') )
finaldf = finaldf.assign(strlen = finaldf["strfeatures"].map(len)) 

#Drops all other columns than the 3(/4) we are interested in.
if "label2" in finaldf.columns:
    finaldf = finaldf[["label", "label2", "strfeatures", "strlen"]]
else:
    finaldf = finaldf[["label", "strfeatures", "strlen"]]


In [11]:
print("finaldf count, before trim:", finaldf.shape[0])
print(finaldf.columns)

# Power apps have a hard limit on 500 JSON objects to be shown. 
# Keep only the first 500 rows
if truncate_down_to_limit:
    finaldf = finaldf.head(truncate_limit)
    print("finaldf count, after trim:", finaldf.shape[0])

finaldf count, before trim: 540
Index(['label', 'strfeatures', 'strlen'], dtype='object')
finaldf count, after trim: 500


In [ ]:
pd.options.display.max_colwidth = 20 #Default = 50

#Check for missing values
errorDF = finaldf.loc[finaldf["label"].isin([None, "B1", "Ukjent banestrekning", "S1", "Ukjent stasjon", "M1"])]
print("Errors are collected in this dataframe:", errorDF[["strfeatures", "strlen"]])

Errors are collected in this dataframe: Empty DataFrame
Columns: [strfeatures, strlen]
Index: []


In [13]:
#Verifying that the convert to circle part works.
#pd.options.display.max_colwidth = 200
#print(finaldf.iloc[finaldf["strfeatures"].str.contains("Circle").values])

In [ ]:
filename = saveFileName
if trainStations:
    filename = filename + "_and_stations"

if truncate_down_to_limit:
    filename = filename + "_trim"
else:
    filename = filename + "_full"

filename = filename + ".xlsx"
finaldf.to_excel(filename)

Eksempel på json_normalize kode som illustrerer konseptet godt.

In [15]:
#Example json_normalize
data = [
    {
        "state": "Florida",
        "shortname": "FL",
        "info": {"governor": "Rick Scott"},
        "counties": [
            {"name": "Dade", "population": 12345},
            {"name": "Broward", "population": 40000},
            {"name": "Palm Beach", "population": 60000},
        ],
    },
    {
        "state": "Ohio",
        "shortname": "OH",
        "info": {"governor": "John Kasich"},
        "counties": [
            {"name": "Summit", "population": 1234},
            {"name": "Cuyahoga", "population": 1337},
        ],
    },
]
print(pd.DataFrame(data))
result = pd.json_normalize(
    data, "counties", [["info", "governor"]])
print("result DF\n", result)

     state shortname                 info             counties
0  Florida        FL  {'governor': 'Ri...  [{'name': 'Dade'...
1     Ohio        OH  {'governor': 'Jo...  [{'name': 'Summi...
result DF
          name  population info.governor
0        Dade       12345    Rick Scott
1     Broward       40000    Rick Scott
2  Palm Beach       60000    Rick Scott
3      Summit        1234   John Kasich
4    Cuyahoga        1337   John Kasich
